In [110]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import requests
from pattern import web
import re, string
from sets import Set
from fuzzywuzzy import fuzz

database = {}

#uh-oh

cardCount = 4972


# convert manacost, cardtext to features


# ratings
# set
# rarity
# artist

# predictors of popularity
# k-means / pca

# FEATURES
# draw x cards
# destroy
# all creature abilities
# damage
# search
# get +1/+1

In [111]:
def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def anyInt(l):
    for val in l:
        if isInt(val): return True
    return False

In [112]:
def grabCard(multiverse_id):
    xml = "http://gatherer.wizards.com/Pages/Card/Details.aspx?multiverseid=" + str(multiverse_id)
    dom = web.Element(requests.get(xml).text)
    
    # card name, card type
    
    cardName = dom('div.cardImage img')[0].attributes['alt'] if dom('div .cardImage img') else ''
        
    cardType = [element.strip() for element in \
                dom('div#ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_typeRow div.value')[0].content.split(u'\u2014')]
    
    # extract, parse, clean text into a list
    
    cardText = []
    for line in dom('div.cardtextbox'):
        for element in line:
            cardText.append(element)
    
    for i in xrange(len(cardText)):
        if cardText[i].type == 'element' and cardText[i].tag == 'img':
            cardText[i] = cardText[i].attributes['alt']
        else:
            cardText[i] = str(cardText[i]).lower()
    
    # mana symbols
    
    manaCost = [element.attributes['alt'] for element in dom('div#ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_manaRow div.value img')]
    cmc = int(dom('div#ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_cmcRow div.value')[0].content.strip()) \
            if dom('div#ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_cmcRow div.value') else np.nan
    
    # p/t
    
    power = np.nan
        
    power = [_.strip() for _ in dom('div#ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow div.value')[0].content.split(' / ')][0] \
                if dom('div#ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow div.value') else np.nan
    power = float(power) if power != '*' and power != np.nan else np.nan
    
    toughness = [_.strip() for _ in dom('div#ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow div.value')[0].content.split(' / ')][1] \
                    if dom('div#ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow div.value') else np.nan
    toughness = float(toughness) if (toughness != '*' and toughness != np.nan) else np.nan
      
        
    # add data
    
    database[cardName] = {
                            'cardType' : cardType,
                            'cardText' : cardText,
                            'manaCost' : manaCost,
                            'cmc' : cmc,
                            'power' : power,
                            'toughness' : toughness
                         }
    

In [113]:
def combineVersions(multiverse_id):
    return

In [114]:
# create database
for i in xrange(1, 100):
    print i
    grabCard(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [154]:
data = pd.DataFrame.from_dict(database, orient='index')

# derive mana features

# card colors

colorlessMana = []
colorless = []

for row in data['manaCost']:
    found = 0
    for val in row:
        if isInt(val):
            colorlessMana.append(float(val))
            found = 1
    if found == 0:
        colorlessMana.append(0)

data['colorlessMana'] = colorlessMana 
data['Variable Colorless'] = [1 if 'Variable Colorless' in text else 0 for text in data['manaCost']]

# count mana symbols

manaSymbols = []

manaSymbols = ['Blue', 'Black', 'Red', 'Green', 'White']
manaVars = ['mana_' + _ for _ in manaSymbols]

for i in xrange(len(manaSymbols)):
    data[manaVars[i]] = [text.count(manaSymbols[i]) for text in data['manaCost']]
    data[manaSymbols[i]] = [1 if text.count(manaSymbols[i]) > 0 else 0 for text in data['manaCost']]

# check colorless

def isColorless(l):
    for val in l:
        if val in manaSymbols: return False
    return True

data['Colorless'] = [1 if isColorless(x) else 0 for x in data['manaCost']]

KeyError: u'Air Elemental'

In [136]:
data.describe()
data

,toughness,power,cmc,cardType,cardText,manaCost,colorlessMana,Variable Colorless,mana_Blue,Blue,mana_Black,Black,mana_Red,Red,mana_Green,Green,mana_White,White,Colorless
Air Elemental,4,4,5,"[Creature, Elemental]",[flying],"[3, Blue, Blue]",3,0,2,1,0,0,0,0,0,0,0,0,0
Ancestral Recall,NaN,NaN,1,[Instant],[target player draws three cards.],[Blue],0,0,1,1,0,0,0,0,0,0,0,0,0
Animate Artifact,NaN,NaN,4,"[Enchantment, Aura]","[enchant artifact, as long as enchanted artifa...","[3, Blue]",3,0,1,1,0,0,0,0,0,0,0,0,0
Animate Dead,NaN,NaN,2,"[Enchantment, Aura]","[enchant creature card in a graveyard, when an...","[1, Black]",1,0,0,0,1,1,0,0,0,0,0,0,0
Ankh of Mishra,NaN,NaN,2,[Artifact],"[whenever a land enters the battlefield, ankh ...",[2],2,0,0,0,0,0,0,0,0,0,0,0,1
Bad Moon,NaN,NaN,2,[Enchantment],[black creatures get +1/+1.],"[1, Black]",1,0,0,0,1,1,0,0,0,0,0,0,0
Basalt Monolith,NaN,NaN,3,[Artifact],[basalt monolith doesn't untap during your unt...,[3],3,0,0,0,0,0,0,0,0,0,0,0,1
Black Knight,2,2,2,"[Creature, Human Knight]","[first strike , <i>(this creature deals combat...","[Black, Black]",0,0,0,0,2,1,0,0,0,0,0,0,0
Black Lotus,NaN,NaN,0,[Artifact],"[Tap, , sacrifice black lotus: add three mana ...",[0],0,0,0,0,0,0,0,0,0,0,0,0,1
Black Vise,NaN,NaN,1,[Artifact],"[as black vise enters the battlefield, choose ...",[1],1,0,0,0,0,0,0,0,0,0,0,0,1


In [146]:
def partialMatch(s, l, threshold=95):
    fuzzVals = [fuzz.partial_ratio(s, x) for x in l]
    if not fuzzVals: fuzzVals = [0]
    return max(fuzzVals) >= threshold

# type features

data['Primary Type'] = [cardType[0] for cardType in data['cardType']]
data = pd.concat([data, pd.get_dummies(data['Primary Type'])], axis=1)

# derive card text/functional features

data['Tap ability'] = [1 if 'Tap' in x else 0 for x in data['cardText']]
data['Mana symbol'] = [1 if anyInt(x) else 0 for x in data['cardText']]
data['Tutor'] = [1 if partialMatch('search your library', l) else 0 for l in data['cardText']]
data['Mana generator'] = [1 if partialMatch('add mana', l) | partialMatch('your mana pool', l) \
                              else 0 for l in data['cardText']]

#MORE

# keyword features

creatureAbilities = ['deathtouch', 'defender', 'first strike', 'double strike', 'enchant', 'equip', \
 'flash', 'flying', 'haste', 'hexproof', 'indestructible', 'intimidate', 'islandwalk', \
 'swampwalk', 'moutainwalk', 'plainswalk', 'forestwalk', 'lifelink', 'protection', 'reach', \
 'shroud', 'trample', 'vigilance']

for ability in creatureAbilities:
    data[ability] = [1 if ability in text else 0 for text in data['cardText']]

# data = pd.concat([data, pd.get_dummies(data['Primary Type'])], axis=1)
 
# ACTION KEYWORDS
    
#  2.1 Attach
# 2.2 Counter
# 2.3 Exile
# 2.4 Fight
# 2.5 Regenerate
# 2.6 Sacrifice
# 2.7 Tap/Untap
 
# ability words
 
#  4.1 Battalion
# 4.2 Bloodrush
# 4.3 Channel
# 4.4 Chroma
# 4.5 Domain
# 4.6 Fateful hour
# 4.7 Ferocious
# 4.8 Grandeur
# 4.9 Hellbent
# 4.10 Heroic
# 4.11 Imprint
# 4.12 Join forces
# 4.13 Kinship
# 4.14 Landfall
# 4.15 Metalcraft
# 4.16 Morbid
# 4.17 Radiance
# 4.18 Raid
# 4.19 Sweep
# 4.20 Threshold
 
#  expert keywords
 
#  3.1 Absorb
# 3.2 Affinity
# 3.3 Amplify
# 3.4 Annihilator
# 3.5 Aura swap
# 3.6 Banding
# 3.7 Bands with other
# 3.8 Battle cry
# 3.9 Bestow
# 3.10 Bolster
# 3.11 Bloodthirst
# 3.12 Bushido
# 3.13 Buyback
# 3.14 Cascade
# 3.15 Champion
# 3.16 Changeling
# 3.17 Cipher
# 3.18 Clash
# 3.19 Conspire
# 3.20 Convoke
# 3.21 Cumulative upkeep
# 3.22 Cycling
# 3.23 Dash
# 3.24 Delve
# 3.25 Detain
# 3.26 Devour
# 3.27 Dredge
# 3.28 Echo
# 3.29 Entwine
# 3.30 Epic
# 3.31 Evolve
# 3.32 Evoke
# 3.33 Exalted
# 3.34 Exploit
# 3.35 Extort
# 3.36 Fading
# 3.37 Fateseal
# 3.38 Fear
# 3.39 Flanking
# 3.40 Flashback
# 3.41 Flip
# 3.42 Forecast
# 3.43 Fortify
# 3.44 Frenzy
# 3.45 Graft
# 3.46 Gravestorm
# 3.47 Haunt
# 3.48 Hideaway
# 3.49 Horsemanship
# 3.50 Infect
# 3.51 Kicker
# 3.52 Level up
# 3.53 Living weapon
# 3.54 Madness
# 3.55 Manifest
# 3.56 Miracle
# 3.57 Modular
# 3.58 Monstrosity
# 3.59 Morph
# 3.60 Multikicker
# 3.61 Ninjutsu
# 3.62 Offering
# 3.63 Overload
# 3.64 Persist
# 3.65 Phasing
# 3.66 Poisonous
# 3.67 Populate
# 3.68 Proliferate
# 3.69 Provoke
# 3.70 Prowl
# 3.71 Rampage
# 3.72 Rebound
# 3.73 Recover
# 3.74 Reinforce
# 3.75 Replicate
# 3.76 Retrace
# 3.77 Ripple
# 3.78 Scavenge
# 3.79 Scry
# 3.80 Shadow
# 3.81 Soulbond
# 3.82 Soulshift
# 3.83 Splice
# 3.84 Split second
# 3.85 Storm
# 3.86 Sunburst
# 3.87 Suspend
# 3.88 Totem armor
# 3.89 Transfigure
# 3.90 Transform
# 3.91 Transmute
# 3.92 Typecycling
# 3.93 Undying
# 3.94 Unearth
# 3.95 Unleash
# 3.96 Vanishing
# 3.97 Wither

In [160]:
data[:['Living Wall']

KeyError: 'Living Wall'

In [120]:
data.to_csv('data.xlsx')